In [ ]:
pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
model1 =EfficientNetB0(include_top=False,input_shape=(224, 224, 3), weights='imagenet')

In [ ]:
input_shape = (224,224)

In [ ]:
conv_model = Model(inputs=model1.input, outputs=model1.output)

In [ ]:
from tensorflow.keras.models import Sequential

new_model = Sequential()

new_model.add(conv_model)

new_model.add(Flatten())

new_model.add(Dense(1024, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(512, activation='relu'))
new_model.add(Dense(2, activation='softmax'))

In [ ]:
batch_size=8

In [ ]:
new_model.build(input_shape = (None,224, 224, 3))
new_model.load_weights('C:/Users/DELL/Desktop/Brain22/BDT/CSMODEL/efficient.h5')

In [ ]:
pip install lime

In [ ]:
import lime
from lime import lime_image
from PIL import Image
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries

def xAI(model,img_dir):
    image = Image.open(img_dir)
    imgx = np.asarray(image)
    #print(imgx[:,:,0].shape)
    #image = image.resize((224, 224))
    img = np.asarray(imgx[:,:,0])
    img = img/255
    img2 = np.zeros( ( np.array(img).shape[0], np.array(img).shape[1], 3 ) )
    img2[:,:,0] = img # same value in each channel
    img2[:,:,1] = img
    img2[:,:,2] = img
    #print(img2.shape)
    explainer = lime_image.LimeImageExplainer(random_state=42)
    explanation = explainer.explain_instance(img2, new_model.predict, labels=["Non-Tumor", "Tumor"],top_labels=2, hide_color=0,num_samples=1000)
    #plt.imshow(img)
    image, mask = explanation.get_image_and_mask(
             model.predict(img2.reshape((1,224,224,3))).argmax(axis=1)[0],
        
             positive_only=False,
        #num_features=10,
             hide_rest=False)
    plt.axis ('off')
    plt.imshow(mark_boundaries(image, mask))


In [ ]:
mod='CSEfficient'

In [ ]:
xAI(new_model, "C:/Users/DELL/Desktop/Brain22/BDT/test/false/No16 resized.png")
plt.savefig('C:/Users/DELL/Desktop/Brain22/BDT/test/0_4_'+mod+'.png')
plt.savefig('C:/Users/DELL/Desktop/Brain22/BDT/test/0_4_'+mod+'.pdf')